In [1]:
import os
os.chdir('/home')

from lib import labels

FINDING: 0.4% of labels have no associated text

maybe we can infer these later by using the graph and the vector space?

In [2]:
word_to_glove = labels.load_embedding()
X = labels.embedding_matrix('train', 20, word_to_glove)
A = labels.adjacency_matrix('train', 20)
print(X.shape, A.shape)

Starting load_embedding...
load_embedding took 47.1388475894928 seconds
(41, 300) (31, 31)
